In [ ]:
import numpy as np
import pandas as pd
import pickle
import csv
import os
import torch
from torch_geometric.data import Data
from tqdm import tqdm
import networkx as nx
from glob import glob
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm
from scipy import sparse
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv, SAGEConv, SGConv

import matplotlib.pyplot as plt
from scipy import sparse
np.random.seed(42)

In [ ]:
def get_graph(path):
    mat = sparse.load_npz(path)
    mat_coo = mat.tocoo()
    mat_csr = mat_coo.tocsr()
    G_new = nx.Graph(mat_csr)
    return G_new
    

In [ ]:
def get_overall_graph_feature(G_new):
    degree_list = []
#     for s in G_new.degree():
#         degree_list.append(s[1])
#     print("degree calculation completed")
    G_new_dc=nx.degree_centrality(G_new)
    # # print(G_new_dc)
    print("degree centrality calculation completed")
    G_new_bc=nx.betweenness_centrality(G_new)
    # print(G_new_bc)
    print("betweeness centrality calculation completed")
    G_new_lc=nx.closeness_centrality(G_new)
    # print(G_new_lc)
    print("closeness centrality calculation completed")
    G_new_ec=nx.eigenvector_centrality(G_new)
    # print(G_new_ec)
    print("Eigen value centrality calculation completed")
    G_Page_rank = nx.pagerank(G_new, alpha=0.9)
    # print(G_Page_rank)
    print("Page rank calculation completed")
    total_feature=[]
    for i in range(G_new.number_of_nodes()):
      feature=[G_new_dc[i],G_new_bc[i],G_new_lc[i],G_new_ec[i],G_Page_rank[i]]
      total_feature.append(feature)
    return total_feature
  

In [ ]:
def save_total_feature(path,total_feature):
    try:
        total_feature_data = open(path,'wb')
        pickle.dump(total_feature,total_feature_data)
        total_feature_data.close()
        print("save suceeded")
    except Exception as e:
        print(e)
        print("total_feature save failed")


In [ ]:
Code_graph = get_graph(r'D:\Andriod_GCN_andVGAEMalGAN\data\processed\code_block_id_tr_reduced.npz')
total_feature=get_overall_graph_feature(Code_graph)




In [ ]:
save_total_feature(r'code_total_feature',total_feature)

In [ ]:
def load_feature(path):
    with open(path, 'rb') as f:
        total_feature = pickle.load(f)
        return total_feature

In [ ]:
training_api = load_feature(r'D:\Andriod_GCN_andVGAEMalGAN\training_api')
api_dictionary = load_feature(r'D:\Andriod_GCN_andVGAEMalGAN\api_dictionary')
reduce_api= load_feature(r'D:\Andriod_GCN_andVGAEMalGAN\reduce_api')

In [ ]:
reduce_api=np.array(reduce_api)
reduce_api

In [ ]:
class YooChooseDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return [r'code_block.dataset']

    def download(self):
        pass
        
    def process(self):
#         pass
                
        data_list = []
        edge_index_path=[r"D:\Andriod_GCN_andVGAEMalGAN\data\interim\class0",r"D:\Andriod_GCN_andVGAEMalGAN\data\interim\class1"]
        count =0
        current=0
        for i in edge_index_path: 
            edge_indexes = glob(f'{i}/*B.npy')
            for edge_index_path in edge_indexes:
                outfile = os.path.basename(edge_index_path)
                outfile=outfile.replace(".B.npy",".npz")
#                 print(outfile)
                current=current+1
                print(current)
                if count == 0:
                    y=[0]
                    save_path=r"D:\Andriod_GCN_andVGAEMalGAN\data\interim\class0\\"
                else:
                    y=[1]
                    save_path=r"D:\Andriod_GCN_andVGAEMalGAN\data\interim\class1\\"
                y = torch.tensor(y)
                print(y)
                edge_index = np.load(f'{edge_index_path}')
                mask_B = np.nonzero(np.isin(edge_index, training_api).sum(axis=0) == 2)[0]
                edge_index = edge_index[:, mask_B]
                #edge_index = replace_with_dict(edge_index, api_dictionary)
                k = np.array(list(api_dictionary.keys()))
                #value is the new api
                v = np.array(list(api_dictionary.values()))
                # Get argsort indices
                sidx = k.argsort()
                ks = k[sidx]
                vs = v[sidx]
                edge_index_processed=vs[np.searchsorted(ks, edge_index)]
                edge_index_processed = np.hstack([edge_index_processed, edge_index_processed[::-1, :]])
                arr = np.unique(edge_index_processed, axis=1)
                values = np.full(shape=edge_index_processed.shape[1], fill_value=1, dtype='i1')
#                 try:
                sparse_arr = sparse.coo_matrix(
                    (values, (edge_index_processed[0], edge_index_processed[1])), shape=(len(training_api),len(training_api))
                ).tocsc()

                edge_index_processed = sparse_arr[reduce_api, :][:, reduce_api] 
                edge_index_processed=edge_index_processed.tocoo()
                edge_index_processed.setdiag(0)
                edge_index_processed.eliminate_zeros()
                sparse.save_npz(os.path.join(save_path+outfile), edge_index_processed)
                edge_index = torch.tensor([edge_index_processed.row.tolist(),edge_index_processed.col.tolist()], dtype=torch.long)
                x = torch.tensor(total_feature,dtype=torch.float)
                data = Data(x=x, edge_index=edge_index, y=y)
                data_list.append(data)
#                 except Exception as e:
#                     print(e)
#                     print(edge_index_path)
            count=count+1
                   
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
custom_dataset= YooChooseDataset(root=r'D:\Andriod_GCN_andVGAEMalGAN\dataset_graph')

In [ ]:
print(f'Dataset: {custom_dataset}:')
print('====================')
print(f'Number of graphs: {len(custom_dataset)}')
print(f'Number of features: {custom_dataset.num_features}')
print(f'Number of classes: {custom_dataset.num_classes}')

data = custom_dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

In [ ]:
torch.manual_seed(12345)
dataset = custom_dataset.shuffle()

train_dataset = dataset[:3]
test_dataset = dataset[3:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

In [ ]:
from torch_geometric.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=62, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=62, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(5, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        emb =x 
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x,emb

model = GCN(hidden_channels=64)
print(model)

In [ ]:
model = GCN(hidden_channels=32)
# model.load_state_dict(torch.load(r"C:\Users\Yumlum\Desktop\Experiment Result\GCNModellocalglobal.pt"))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#enable only when testing
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
#enable only during testing
def get_embedding(data):
    model.eval()
    data=data.to(device)
    out,emb = model(data.x, data.edge_index, data.batch)  
    pred = out.argmax(dim=1)
#     pred_numpy = pred.cpu().numpy()
#     print(pred)
    return pred,emb

In [ ]:
def train():
    model.train()
    
    running_loss = 0.0
    for data in train_loader:  # Iterate in batches over the training dataset.
         data.x = data.x.to(device)
         data.edge_index= data.edge_index.to(device)
         data.batch=data.batch.to(device)
         data.y = data.y.to(device)
         out,emb = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad() # Clear gradients.
         running_loss += loss.item() * data.num_graphs
    running_loss /= len(train_loader.dataset)
    return running_loss

def test(loader):
     model.eval()
     correct = 0
     running_loss = 0.0
     prediction=[]
     truelabel=[]
     for data in loader:  # Iterate in batches over the training/test dataset.
         data=data.to(device)
         out,emb = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         pred_numpy = pred.cpu().numpy()
         true_label = data.y.cpu().numpy()
         loss = criterion(out, data.y)
         running_loss += loss.item() * data.num_graphs
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
         prediction.append(pred_numpy)
         truelabel.append(true_label)
     running_loss /= len(train_loader.dataset)
     return correct / len(loader.dataset),running_loss,prediction,truelabel  # Derive ratio of correct predictions.
trainlosses = []
testlosses=[]
prediction=[]
truelabel=[]
for epoch in range(1, 300):
    loss=train()
    train_acc,train_loss,train_pred_numpy,train_true_label = test(train_loader)
    trainlosses.append(train_loss)
    test_acc,test_loss,pred_numpy,true_label = test(test_loader)
    testlosses.append(test_loss)
    prediction.append(pred_numpy)
    truelabel.append(true_label)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f},Train loss:{train_loss:.4f},Test loss:{test_loss:.4f}')
   

In [ ]:
#enable only during testing
edge_index_path=[r"D:\Andriod_GCN_andVGAEMalGAN\data\interim\class0",r"D:\Andriod_GCN_andVGAEMalGAN\data\interim\class1"]
with open(r'embedding.csv', 'w') as f:
    write = csv.writer(f) 
    count=0
    index=0
    prediction=[]
    truelabel=[]
    for i in edge_index_path:      
        if count==0:
            Type=0
        if count==1:
            Type=1
        edge_indexes = glob(f'{i}/*.npz') 
        for j in edge_indexes:
            try:
                base_name=os.path.basename(j).replace(".npz","")
                edge_index = sparse.load_npz(j)
                edge_index = torch.tensor([edge_index.row.tolist(),edge_index.col.tolist()], dtype=torch.long)  
    #                 features=[]
    #                 for i,j in zip(code_block_total_feature,local_centrality):
    #                     features.append(i+j)
                x = torch.tensor(total_feature,dtype=torch.float)
                data = Data(x=x, edge_index=edge_index)
                data.batch=torch.zeros(8781, dtype=torch.long)
                pred,embedding=get_embedding(data)
                pred_numpy = pred.cpu().numpy()
                embs=embedding.cpu()
                embedding=embs.detach().numpy().tolist()
                prediction.append(pred_numpy)
                truelabel.append(Type)
                write.writerow([base_name]+embedding[0]+[Type])
                index=index+1
            except Exception as e:
                print(e)
                print("failed")
                print(j)
        count=count+1

In [ ]:
predictionnew_test=[l.tolist() for l in prediction]
predictionlist_test = [item for sublist in predictionnew_test for item in sublist]
print(predictionlist_test)

In [ ]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,accuracy_score

In [ ]:
print("Recall",recall_score(truelabel, predictionlist_test))
print("precision",precision_score(truelabel, predictionlist_test))
print("f1 score",f1_score(truelabel, predictionlist_test))
print("accuracy score",accuracy_score(truelabel, predictionlist_test))
confusion_matrix(truelabel, predictionlist_test)

In [ ]:
# fig = plt.figure()
# plt.plot(epochs, trainlosses, 'g', label='Training loss')
# plt.plot(epochs, testlosses, 'b', label='validation loss')
# plt.title('Training and Validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [ ]:
from torch_geometric.nn import GraphConv


class GraphSage(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GraphSage, self).__init__()
        torch.manual_seed(12345)
        self.conv1 =  SAGEConv(dataset.num_node_features, hidden_channels)
        self.conv2 =  SAGEConv(hidden_channels, hidden_channels)
        self.conv3 =  SAGEConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        emb =x 
        x = global_mean_pool(x, batch)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x,emb

model = GraphSage(hidden_channels=32)
print(model)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = GraphSage(hidden_channels=32)
print(model)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
def train():
    model.train()
    
    running_loss = 0.0
    for data in train_loader:  # Iterate in batches over the training dataset.
         data.x = data.x.to(device)
         data.edge_index= data.edge_index.to(device)
         data.batch=data.batch.to(device)
         data.y = data.y.to(device)
         out,emb = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad() # Clear gradients.
         running_loss += loss.item() * data.num_graphs
    running_loss /= len(train_loader.dataset)
    return running_loss

def test(loader):
     model.eval()
     correct = 0
     running_loss = 0.0
     prediction=[]
     truelabel=[]
     for data in loader:  # Iterate in batches over the training/test dataset.
         data=data.to(device)
         out,emb = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         pred_numpy = pred.cpu().numpy()
         true_label = data.y.cpu().numpy()
         loss = criterion(out, data.y)
         running_loss += loss.item() * data.num_graphs
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
         prediction.append(pred_numpy)
         truelabel.append(true_label)
     running_loss /= len(train_loader.dataset)
     return correct / len(loader.dataset),running_loss,prediction,truelabel  # Derive ratio of correct predictions.

for epoch in range(1, 200):
    loss=train()
    train_acc,train_loss,train_pred_numpy,train_true_label = test(train_loader)
    trainlosses.append(train_loss)
    test_acc,test_loss,pred_numpy,true_label = test(test_loader)
    testlosses.append(test_loss)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f},Train loss:{train_loss:.4f},Test loss:{test_loss:.4f}')
#     if train_acc>=0.93 and test_acc >=0.93 :
#         break

In [ ]:
#enable only during testing
edge_index_path=[r"D:\Andriod_GCN_andVGAEMalGAN\data\interim\class0",r"D:\Andriod_GCN_andVGAEMalGAN\data\interim\class1"]
with open(r'embeddingsage.csv', 'w') as f:
    write = csv.writer(f) 
    count=0
    index=0
    prediction=[]
    truelabel=[]
    for i in edge_index_path:      
        if count==0:
            Type=0
        if count==1:
            Type=1
        edge_indexes = glob(f'{i}/*.npz') 
        for j in edge_indexes:
            try:
                base_name=os.path.basename(j).replace(".npz","")
                edge_index = sparse.load_npz(j)
                edge_index = torch.tensor([edge_index.row.tolist(),edge_index.col.tolist()], dtype=torch.long)  
    #                 features=[]
    #                 for i,j in zip(code_block_total_feature,local_centrality):
    #                     features.append(i+j)
                x = torch.tensor(total_feature,dtype=torch.float)
                data = Data(x=x, edge_index=edge_index)
                data.batch=torch.zeros(8781, dtype=torch.long)
                pred,embedding=get_embedding(data)
                pred_numpy = pred.cpu().numpy()
                embs=embedding.cpu()
                embedding=embs.detach().numpy().tolist()
                prediction.append(pred_numpy)
                truelabel.append(Type)
                write.writerow([base_name]+embedding[0]+[Type])
                index=index+1
            except Exception as e:
                print(e)
                print("failed")
                print(j)
        count=count+1